# Perryman 1998

Differences:
1.  This work will be in equitorial coordinates
2.  No projection  of center motion to candidate members since cluster is so far away at 2kpc
3.  Working only with tangental velocity

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#matplotlib inline

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

import sys
sys.path.append('./src')

from data_queries import  getClusterInfo, getGAIAKnownMembers
from coeval import coeval
from gaiastars import gaiastars as gs,gaiadr2xdr3

import astropy.units as u
from astropy.coordinates import SkyCoord

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
    #trumpler meta data
    trumpler_df = pd.DataFrame([
        ['Trumpler14', '10:43:55.4','-59:32:16', 2.37,0.15, 264, -6.58, 0.06, 2.185, 0.084],
        ['Trumpler15', '10:44:40.8', '-59:22:10', 2.36, 0.09, 320, np.nan, np.nan, np.nan, np.nan],
        ['Trumpler16', '10:45:10.6', '-59:42:28', 2.32,0.12, 320, -6.931,0.063, 2.612, 0.058]
    ], columns=['ClusterName','ra', 'dec','distance','disterr','radius','pm_ra_cosdec','pm_ra_cosdec_error','pm_dec','pm_dec_error']
    ).set_index('ClusterName')
    tc = SkyCoord(ra=trumpler_df.ra, dec=trumpler_df.dec,
        pm_ra_cosdec = list(trumpler_df.pm_ra_cosdec)*u.mas/u.year,
        pm_dec = list(trumpler_df.pm_dec)*u.mas/u.year,
        unit=(u.hourangle, u.deg),
        distance = list(trumpler_df.distance)*u.kpc)
    
    trumpler_coords = {}
    for i, cl in enumerate(trumpler_df.index):
        trumpler_coords[cl]=tc[i]


In [3]:
trumpler_coords['Trumpler15']

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    (161.17, -59.36944444, 2.36)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (nan, nan)>

In [4]:
carina_known_members = pd.read_csv('./data/carina_members.csv', comment='#')

In [5]:
errorcolumns = [
    'ra_error', 'dec_error', 'parallax_error', 'pmra_error', 'pmdec_error','dr2_radial_velocity_error',
    'ra_dec_corr', 'ra_parallax_corr','ra_pmra_corr', 'ra_pmdec_corr',
    'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr',
    'parallax_pmra_corr', 'parallax_pmdec_corr',
    'pmra_pmdec_corr'
]
#add table to query to get the ruwe parameter
fixme = gs(name='fixme')
fixme.add_table_columns(errorcolumns)

In [6]:

carina_members={}
#for cl in cluster_names:
for cl, cluster in carina_known_members.groupby('Cluster'):
    known_members = list(cluster['Gaia Number'])
    print(f'Fetching {cl}')
    carina_members[cl]  = gs(name = cl, description=f'{cl} sources from Shull table from Gaia eDR3')
    carina_members[cl].from_source_idlist(known_members,schema='gaiaedr3', query_type='sync')

Fetching Bochum11
Fetching Other
Fetching Trumpler14
Fetching Trumpler15
Fetching Trumpler16


In [7]:
carina_members[cl].objs.columns

Index(['source_id', 'ra', 'dec', 'parallax', 'pmra', 'pmdec',
       'radial_velocity', 'phot_g_mean_mag', 'phot_bp_mean_mag',
       'phot_rp_mean_mag', 'ruwe', 'ra_error', 'dec_error', 'parallax_error',
       'pmra_error', 'pmdec_error', 'radial_velocity_error', 'ra_dec_corr',
       'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr',
       'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr',
       'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr',
       'r_est'],
      dtype='object')

In [8]:
#carina_cluster_names = list(carina_members.keys())
# can only deal with the Trumplers at the momemnt
carina_cluster_names = ['Trumpler14', 'Trumpler15', 'Trumpler16']

In [9]:
for cl in carina_cluster_names:

    center=trumpler_coords[cl]
    coords = carina_members[cl].get_coords()
    cen_dists = center.separation_3d(coords).to_value(u.pc)
    cen_seps = center.separation(coords).to_value(u.arcsecond)
    carina_members[cl].objs['DistanceFromCenter'] = cen_dists
    carina_members[cl].objs['SeparationFromCenter'] = cen_seps

In [10]:
print('\n--- Min/Max distance of Known Members from Cluster Center ---\n')
for cl in carina_cluster_names:
    print(f'Cluster: {cl}, Min distance: {carina_members[cl].objs.DistanceFromCenter.min():.2f} pc, Max distance: {carina_members[cl].objs.DistanceFromCenter.max():.2f} pc' )


--- Min/Max distance of Known Members from Cluster Center ---

Cluster: Trumpler14, Min distance: 11.60 pc, Max distance: 685.98 pc
Cluster: Trumpler15, Min distance: 23.14 pc, Max distance: 161.58 pc
Cluster: Trumpler16, Min distance: 3.68 pc, Max distance: 390.31 pc


In [11]:
print('\n--- Min/Max distance of Known Members from Cluster Center ---\n')
for cl in carina_cluster_names:
    print(f'Cluster: {cl}, Min Separation: {carina_members[cl].objs.SeparationFromCenter.min():.2f} as, Max Separation: {carina_members[cl].objs.SeparationFromCenter.max():.2f} as' )


--- Min/Max distance of Known Members from Cluster Center ---

Cluster: Trumpler14, Min Separation: 32.84 as, Max Separation: 322.12 as
Cluster: Trumpler15, Min Separation: 50.63 as, Max Separation: 1248.25 as
Cluster: Trumpler16, Min Separation: 43.13 as, Max Separation: 910.22 as


In [12]:
from gaiastars import from_pickle

carina_search_results = from_pickle(f'./data/carina_search_results')

In [13]:
# compute distance relative to Trumpler 15
cl='Trumpler15'

center=trumpler_coords[cl]
coords = carina_search_results.get_coords()
cen_dists = center.separation_3d(coords).to_value(u.pc)
cen_seps = center.separation(coords).to_value(u.arcsecond)
carina_search_results.objs['DistanceFromCenter'] = cen_dists
carina_search_results.objs['SeparationCenter']= cen_seps
# known members
coords = carina_members[cl].get_coords()
cen_dists = center.separation_3d(coords).to_value(u.pc)
cen_seps = center.separation(coords).to_value(u.arcsecond)
carina_members[cl].objs['DistanceFromCenter'] = cen_dists
carina_members[cl].objs['SeparationCenter']= cen_seps


In [14]:
from src.perryman import perryman

In [15]:
t14 = perryman(carina_search_results.objs, list(carina_members['Trumpler14'].objs.index))

Model initialized with 16 members; Missing members: 4
calculating velocities and errors
Tangental Velocity Shape: (107081, 2, 1)
Covariance.shape: (107081, 3, 3)
Tangental_velocity_error_covar.shape: (107081, 2, 2)
R.shape: (107081, 3, 2)
objs_dxyz.shape: (107081, 3, 1)
objs_xyz.shape: (107081, 3, 1)
objs_dxyz_covar.shape: (107081, 3, 3)


In [16]:
from perryman import get_R, eq_to_cartesian, get_pm_jacobian, cartesian_to_eq

## Some Validation Helper Routines

In [17]:
import re
def covar_to_err_corr(covar, labs):
    err = np.sqrt(np.diag(covar))
    dinv = np.diag(1/err)
    corr = dinv.dot(covar.dot(dinv))
    n = len(labs)
    
    corr_strs = ['{l1}_{l2}_corr'.format(l1=labs[i].replace('_error',''),
                                               l2=labs[j].replace('_error',''))\
                 for i in range(n) for j in range(i+1,n)]
    corrs = [corr[i,j] for i in range(n) for j in range(i+1,n)]
    #print(corr_strs)
    
    errdict = {**{l:err[i] for i,l in enumerate(labs)}, **{l:c for l,c in zip(corr_strs, corrs)}}
    return errdict

## Let the Validation Begin!
First, let's validate some of the calculations

### Spherical to Cartesian

In [18]:
#search_results.objs and t14.objs assumed to be isomorphic
t14_coords = carina_search_results.get_coords()
star_i = 34334
s = t14.objs.iloc[star_i]
xyz, d_xyz, d_xyz_covar,pm_error, pm_covar, tang_v, tang_v_covar, pm_jacobian, R = eq_to_cartesian(None,s)
xyz, d_xyz, d_xyz_covar

(array([[-1115.40290872],
        [  363.73769014],
        [-1966.87408371]]),
 array([[ 1.21096096],
        [83.47911333],
        [14.75121906]]),
 array([[ 0.11765386,  0.68051986,  0.05912898],
        [ 0.68051986, 43.94395143,  7.74071797],
        [ 0.05912898,  7.74071797,  1.3979737 ]]))

In [19]:
t14_coords[star_i].cartesian.xyz.value.reshape(-1,1)

array([[-1115.40290872],
       [  363.73769014],
       [-1966.87408371]])

In [20]:
np.allclose(t14_coords[star_i].cartesian.xyz.value.reshape(-1,1),xyz)

True

In [21]:
t14_coords[star_i].velocity.d_xyz.value.reshape(-1,1)

array([[ 1.21096096],
       [83.47911333],
       [14.75121906]])

In [22]:
np.allclose(t14_coords[star_i].velocity.d_xyz.value.reshape(-1,1),d_xyz)

True

Transformations from spherical coordinates to cartesian agree with Astropy. **!YAY!**

### Retrieve Motion Error & Correlations from Motion Covariance Matrix

In [23]:
s

ra                        161.938630
dec                       -59.184512
parallax                    0.436643
pmra                       -7.344922
pmdec                       2.652342
radial_velocity                  NaN
phot_g_mean_mag            15.802890
phot_bp_mean_mag           16.345383
phot_rp_mean_mag           15.052031
ruwe                        1.144181
ra_error                    0.030158
dec_error                   0.026451
parallax_error              0.034282
pmra_error                  0.038519
pmdec_error                 0.032923
radial_velocity_error            NaN
ra_dec_corr                 0.065900
ra_parallax_corr           -0.109937
ra_pmra_corr               -0.114580
ra_pmdec_corr               0.057084
dec_parallax_corr           0.061034
dec_pmra_corr               0.061835
dec_pmdec_corr             -0.055852
parallax_pmra_corr          0.145616
parallax_pmdec_corr        -0.054600
pmra_pmdec_corr             0.080253
r_est                    2290.201392
e

In [24]:
pm_covar

array([[ 1.17522148e-03,  1.92285773e-04, -6.16239593e-05,
         0.00000000e+00],
       [ 1.92285773e-04,  1.48373680e-03,  1.01773103e-04,
         0.00000000e+00],
       [-6.16239593e-05,  1.01773103e-04,  1.08389657e-03,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [25]:
covar_to_err_corr(pm_covar,labs=['parallax_error','pmra_error','pmdec_error','radial_velocity_error'])

/home/kevin_leo_mcmanus/anaconda3/envs/astro/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


{'parallax_error': 0.03428150340914726,
 'pmra_error': 0.03851930424571037,
 'pmdec_error': 0.03292258456349373,
 'radial_velocity_error': 0.0,
 'parallax_pmra_corr': 0.14561592042446136,
 'parallax_pmdec_corr': -0.05460039898753166,
 'parallax_radial_velocity_corr': nan,
 'pmra_pmdec_corr': 0.08025288581848143,
 'pmra_radial_velocity_corr': nan,
 'pmdec_radial_velocity_corr': nan}

### Tangental Velocity and Error

In [26]:
tang_v, tang_v_covar

(array([[-79.74111908],
        [ 28.79550625],
        [  0.        ]]),
 array([[ 40.13245737, -14.40168944,   0.        ],
        [-14.40168944,   5.32712162,   0.        ],
        [  0.        ,   0.        ,   0.        ]]))

In [27]:
covar_to_err_corr(tang_v_covar,['VRA_error','VDec_error','VR'])

/home/kevin_leo_mcmanus/anaconda3/envs/astro/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


{'VRA_error': 6.335018340221204,
 'VDec_error': 2.3080558096518393,
 'VR': 0.0,
 'VRA_VDec_corr': -0.9849615082104841,
 'VRA_VR_corr': nan,
 'VDec_VR_corr': nan}

Notice loss of precision in VRA and VDec compared to PMRA and PMDec. RA goes from $0.034282\ /\ 7.344922 = 0.00467$ to $6.33501834\ /\ 79.74111908 = 0.07944 $

### No Inverse Transform from Tangental Velocity Covariance to PM Covariance

The transformation from Proper Motion errors and correlations to that of Tangental Velocity is given by
$$
C_{tang\_v} = J C_{PM} J^T
$$
where $C_{PM}$ is the covariance matrix derived the errors and correlation on the observed proper motions and $C_{tang\_v}$ is the covariance matrix of of the tangental velocities. $J$ is the Jacobian of the transformation from 
$(\varpi, \mu_{\alpha*}, \mu_\delta, V_R)$ to $(V_{\alpha*}, V_{\delta}, V_R)$ .
$J$ is given in Perryman98 in Equation 15.

In general, $J$ is singular, i.e. not square, so the equation above cannot be inverted.
In the case at hand, $J$ is of dimension $(3,4)$

## Retreive Tangental Velocity Covariance from Space Velocity Covariance

In [28]:
tang_v_covar

array([[ 40.13245737, -14.40168944,   0.        ],
       [-14.40168944,   5.32712162,   0.        ],
       [  0.        ,   0.        ,   0.        ]])

In [29]:
d_xyz_covar

array([[ 0.11765386,  0.68051986,  0.05912898],
       [ 0.68051986, 43.94395143,  7.74071797],
       [ 0.05912898,  7.74071797,  1.3979737 ]])

In [30]:
covar_to_err_corr(d_xyz_covar,labs=['x_error','y_error','z_error'])

{'x_error': 0.34300708838128613,
 'y_error': 6.629023414548858,
 'z_error': 1.182359377723125,
 'x_y_corr': 0.29928720744544957,
 'x_z_corr': 0.14579673515728556,
 'y_z_corr': 0.9876025236568243}

In [31]:
d_xyz

array([[ 1.21096096],
       [83.47911333],
       [14.75121906]])

In [32]:
R.dot(d_xyz_covar.dot(R.T))

array([[ 36.15813944, -11.13822214, -14.39146143],
       [-11.13822214,   3.54940821,   4.37984912],
       [-14.39146143,   4.37984912,   5.75203135]])

In [49]:
np.allclose(tang_v_covar, R.T.dot(d_xyz_covar.dot(R)))

True

Yes, we can retrieve Tangental Velocity Covariance from Space Velocity Covariance because the transformation matrix ($R$) is square $(3,3)$ and thus invertable.

## Validated So Far:

1. Transformation from Equatorial to Cartesian Coords is correct
2. Transfomration from Proper Motion to Space Velocity is correct
3. Inverse transformation from Covariance to standard errors and correlations
4. Inability to transform from Space Velocity Covariance matrix to PM Covariance and thus to PM errors and correlations
    

## Compare to Model Calculations

In [34]:
info = t14.get_info(star_i)

### Position and Velocity

In [35]:
info['R']

array([[-0.31003551, -0.81650301],
       [-0.95072498,  0.26626514],
       [ 0.        ,  0.51227504]])

In [36]:
R

array([[-0.31003551, -0.81650301, -0.48703267],
       [-0.95072498,  0.26626514,  0.15882345],
       [ 0.        ,  0.51227504, -0.85882145]])

In [37]:
np.allclose(info['R'], R[:,:2] )

True

In [38]:
np.allclose(info['space_pos'], xyz)

True

In [39]:
np.allclose(info['space_velocity'],d_xyz)

True

In [40]:
np.allclose(info['tangental_velocity'],tang_v[:2])

True

### Tangental Velocity Covariances

In [41]:
tang_v_covar

array([[ 40.13245737, -14.40168944,   0.        ],
       [-14.40168944,   5.32712162,   0.        ],
       [  0.        ,   0.        ,   0.        ]])

In [42]:
info['tangental_velocity_covar']

array([[ 40.13245737, -14.40168944],
       [-14.40168944,   5.32712162]])

In [43]:
np.allclose(info['tangental_velocity_covar'], tang_v_covar[:2,:2])

True

### Proper Motion Covariances

In [44]:
pm_covar[:3,:3]

array([[ 1.17522148e-03,  1.92285773e-04, -6.16239593e-05],
       [ 1.92285773e-04,  1.48373680e-03,  1.01773103e-04],
       [-6.16239593e-05,  1.01773103e-04,  1.08389657e-03]])

In [45]:
#permute the rows & columns
colperm = [2,0,1]
info['PM_covar'][colperm][:,colperm]

array([[ 1.17522148e-03,  1.92285773e-04, -6.16239593e-05],
       [ 1.92285773e-04,  1.48373680e-03,  1.01773103e-04],
       [-6.16239593e-05,  1.01773103e-04,  1.08389657e-03]])

In [46]:
np.allclose(info['PM_covar'][colperm][:,colperm],pm_covar[:3,:3])

True

### Space Velocity Covariance

In [47]:
info['space_velocity_covar'], d_xyz_covar

(array([[ 0.11765386,  0.68051986,  0.05912898],
        [ 0.68051986, 43.94395143,  7.74071797],
        [ 0.05912898,  7.74071797,  1.3979737 ]]),
 array([[ 0.11765386,  0.68051986,  0.05912898],
        [ 0.68051986, 43.94395143,  7.74071797],
        [ 0.05912898,  7.74071797,  1.3979737 ]]))

In [48]:
np.allclose(info['space_velocity_covar'], d_xyz_covar)

True

All the model set-up calculations check out! **!Yay!**